# Credit Card Approval Prediction


## Context

Credit score cards are a common risk control method in the financial industry. It uses personal information and data submitted by credit card applicants to predict the probability of future defaults and credit card borrowings. The bank is able to decide whether to issue a credit card to the applicant. Credit scores can objectively quantify the magnitude of risk.

Generally speaking, credit score cards are based on historical data. Once encountering large economic fluctuations. Past models may lose their original predictive power. Logistic model is a common method for credit scoring. Because Logistic is suitable for binary classification tasks and can calculate the coefficients of each feature. In order to facilitate understanding and operation, the score card will multiply the logistic regression coefficient by a certain value (such as 100) and round it.

At present, with the development of machine learning algorithms. More predictive methods such as Boosting, Random Forest, and Support Vector Machines have been introduced into credit card scoring. However, these methods often do not have good transparency. It may be difficult to provide customers and regulators with a reason for rejection or acceptance.

## Task

Build a machine learning model to predict if an applicant is 'good' or 'bad' client, different from other tasks, the definition of 'good' or 'bad' is not given. You should use some techique, such as vintage analysis to construct you label. Also, unbalance data problem is a big problem in this task.

In [401]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [402]:
# There are 2 datasets 

# 1. application records (contains all information about the applicants)

application_records = pd.read_csv('../../data/raw/application_record.csv')

# 2. credit records (contains information about the loan and depth)

credit_records = pd.read_csv('../../data/raw/credit_record.csv')

# 1. Exploratory Data Analysis

In [403]:
display(application_records.head())
display(credit_records.head())

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [404]:
# Standardizing the column names for both files: 

def columns(x):
    return x.lower()

application_records.columns = list(map(columns, application_records.columns))
credit_records.columns = list(map(columns, credit_records.columns))


In [405]:
# Checking if the data of the two files match in length: 

print('Application Records = {}'.format(application_records.shape))
print('Credit Records = {}'.format(credit_records.shape))

#There is a big difference in data length in the two files

Application Records = (438557, 18)
Credit Records = (1048575, 3)


## Predicted Feature

**Column Status:** 

- 0: 1-29 days past due
- 1: 30-59 days past due
- 2: 60-89 days overdue
- 3: 90-119 days overdue
- 4: 120-149 days overdue
- 5: Overdue or bad debts, write-offs for more than 150 days
- C: paid off that month
- X: No loan for the month



**Column Months_Balance:**

The month of the extracted data is the starting point, backwards

- 0: current month
- -1: previous month
- ...


In [406]:
print(credit_records.shape)
credit_records['id'].nunique()

(1048575, 3)


45985

In [407]:
credit_records['status'].value_counts()

C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: status, dtype: int64

In [408]:
pivot_tb = pd.pivot_table(credit_records, index = ['id', 'status'], aggfunc = 'count').reset_index()
pivot_tb


,id,status,months_balance
0,5001711,0,3
1,5001711,X,1
2,5001712,0,10
3,5001712,C,9
4,5001713,X,22
...,...,...,...
94138,5150483,X,18
94139,5150484,0,12
94140,5150484,C,1
94141,5150485,0,2


### Labeling the customers

In [409]:
# Find percentage of customers for each status

ratio_overdue = []

# ratio of customers overdue more than 1 day:
overdue0 = round(len(pivot_tb[pivot_tb['status'] == '0']) / pivot_tb['id'].nunique(), 3)
ratio_overdue.append(overdue0)

# ratio of customers overdue more than 30 days:
overdue1 = round(len(pivot_tb[pivot_tb['status'] == '1']) / pivot_tb['id'].nunique(), 3)
ratio_overdue.append(overdue1)

# ratio of customers overdue more than 60 day:
overdue2 = round(len(pivot_tb[pivot_tb['status'] == '2']) / pivot_tb['id'].nunique(), 4)
ratio_overdue.append(overdue2)

# ratio of customers overdue more than 90 day:
overdue3 = round(len(pivot_tb[pivot_tb['status'] == '3']) / pivot_tb['id'].nunique(), 4)
ratio_overdue.append(overdue3)
                     
# ratio of customers overdue more than 120 day:
overdue4 = round(len(pivot_tb[pivot_tb['status'] == '4']) / pivot_tb['id'].nunique(), 5)
ratio_overdue.append(overdue4)

# ratio of customers overdue more than 150 day:
overdue5 = round(len(pivot_tb[pivot_tb['status'] == '5']) / pivot_tb['id'].nunique(), 5)
ratio_overdue.append(overdue5)

ratio_overdue

[0.869, 0.113, 0.013, 0.0058, 0.00413, 0.00424]

In [410]:
# Creating a Dictonary and a Dataframe out of the caculated ratio.

dict = {'default': ['>1', '>30', '>60', '>90', '>120', '>150'], 
       'ratio': ratio_overdue}

pd.DataFrame.from_dict(dict)


,default,ratio
0,>1,0.86900
1,>30,0.11300
2,>60,0.01300
3,>90,0.00580
4,>120,0.00413
5,>150,0.00424


The percentage of customers who are overdue more than 1 day lies by 87%. If we would consider customers with STATUS 0 as RISK Customers this would lead to a lot of people being classified as bad. Those would not receive a loan. However the bank would also loose a lot of profit by being too risk averse. 

Considering the customers with more than 30 day overdue also leads to a relative high amount of customers classified as bad. More than 10% of customers are overdue more than 30days at least once. 

Customers overdue more than 60 days are already at a reasonable percentag of 1.3% 

**---> Customers >= STATUS 2 will be labeled as RISK!** 


In [411]:
# Flag Customers as 1 or 0. 1 = RISK / 0 = NO RISK

pivot_tb['default_flag'] = [0 if i in ['X', '0', '1', 'C'] else 1 for i in pivot_tb['status']] #each Status per customer will be flagged
pivot_tb.head()

# this approach leads to several values per customer -> however I am only interested in weather the customer is classified as riks or no risk. 

,id,status,months_balance,default_flag
0,5001711,0,3,0
1,5001711,X,1,0
2,5001712,0,10,0
3,5001712,C,9,0
4,5001713,X,22,0


In [412]:
# A customer which has one or more month with a status higher than 2 will be classified as RISK

# 1. Compute the sum of all status labels per customer
flagged = pd.pivot_table(pivot_tb, index = 'id', values = 'default_flag', aggfunc = 'sum').reset_index()

# 2. if a customer has on ore more values flagged as 1, the overall lable will be RISK
pivot_tb['default_flag'] = pivot_tb['default_flag'].apply(lambda x: 1 if x >= 1 else 0) #save in a new column

In [413]:
# Checking for plausability of calculation:

pivot_tb['default_flag'].value_counts()

# -- > There is a CLASS IMBALANCE

0    92893
1     1250
Name: default_flag, dtype: int64

## Dependent Features

In [414]:
application_records.head()

,id,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,name_income_type,name_education_type,name_family_status,name_housing_type,days_birth,days_employed,flag_mobil,flag_work_phone,flag_phone,flag_email,occupation_type,cnt_fam_members
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [415]:
# Are the columns in the correct dtype

application_records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   438557 non-null  int64  
 1   code_gender          438557 non-null  object 
 2   flag_own_car         438557 non-null  object 
 3   flag_own_realty      438557 non-null  object 
 4   cnt_children         438557 non-null  int64  
 5   amt_income_total     438557 non-null  float64
 6   name_income_type     438557 non-null  object 
 7   name_education_type  438557 non-null  object 
 8   name_family_status   438557 non-null  object 
 9   name_housing_type    438557 non-null  object 
 10  days_birth           438557 non-null  int64  
 11  days_employed        438557 non-null  int64  
 12  flag_mobil           438557 non-null  int64  
 13  flag_work_phone      438557 non-null  int64  
 14  flag_phone           438557 non-null  int64  
 15  flag_email       

#### Missing Values 

In [416]:
application_records.isna().sum()

id                          0
code_gender                 0
flag_own_car                0
flag_own_realty             0
cnt_children                0
amt_income_total            0
name_income_type            0
name_education_type         0
name_family_status          0
name_housing_type           0
days_birth                  0
days_employed               0
flag_mobil                  0
flag_work_phone             0
flag_phone                  0
flag_email                  0
occupation_type        134203
cnt_fam_members             0
dtype: int64

In [417]:
application_records['occupation_type'].value_counts( dropna = False)

NaN                      134203
Laborers                  78240
Core staff                43007
Sales staff               41098
Managers                  35487
Drivers                   26090
High skill tech staff     17289
Accountants               15985
Medicine staff            13520
Cooking staff              8076
Security staff             7993
Cleaning staff             5845
Private service staff      3456
Low-skill Laborers         2140
Secretaries                2044
Waiters/barmen staff       1665
Realty agents              1041
HR staff                    774
IT staff                    604
Name: occupation_type, dtype: int64

In [418]:
# Fill Missing Values: 

application_records['occupation_type'] = application_records['occupation_type'].fillna('Others')
application_records['occupation_type'].value_counts( dropna = False)

Others                   134203
Laborers                  78240
Core staff                43007
Sales staff               41098
Managers                  35487
Drivers                   26090
High skill tech staff     17289
Accountants               15985
Medicine staff            13520
Cooking staff              8076
Security staff             7993
Cleaning staff             5845
Private service staff      3456
Low-skill Laborers         2140
Secretaries                2044
Waiters/barmen staff       1665
Realty agents              1041
HR staff                    774
IT staff                    604
Name: occupation_type, dtype: int64

### Categorical Values: 

In [419]:
application_cat = application_records.select_dtypes('object')
application_cat.head()

,code_gender,flag_own_car,flag_own_realty,name_income_type,name_education_type,name_family_status,name_housing_type,occupation_type
0,M,Y,Y,Working,Higher education,Civil marriage,Rented apartment,Others
1,M,Y,Y,Working,Higher education,Civil marriage,Rented apartment,Others
2,M,Y,Y,Working,Secondary / secondary special,Married,House / apartment,Security staff
3,F,N,Y,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff
4,F,N,Y,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff


#### Reducing & Mergin Classes

In [420]:
# checking for the amount of classes in each feature
application_cat.nunique()

code_gender             2
flag_own_car            2
flag_own_realty         2
name_income_type        5
name_education_type     5
name_family_status      5
name_housing_type       6
occupation_type        19
dtype: int64

In [421]:
# checking the frequency of classes in INCOME TYPE
application_cat['name_income_type'].value_counts()

Working                 226104
Commercial associate    100757
Pensioner                75493
State servant            36186
Student                     17
Name: name_income_type, dtype: int64

In [422]:
# I will merge 'Student' and 'Pensioner' 

application_cat.loc[(application_cat['name_income_type'] == 'Student') | (application_cat['name_income_type'] == 'Pensioner'), 'name_income_type'] = 'Others'
application_cat['name_income_type'].value_counts()


/Users/steib/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/steib/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Working                 226104
Commercial associate    100757
Others                   75510
State servant            36186
Name: name_income_type, dtype: int64

In [423]:
# checking the frequency of classes in INCOME TYPE
application_cat['name_education_type'].value_counts()

Secondary / secondary special    301821
Higher education                 117522
Incomplete higher                 14851
Lower secondary                    4051
Academic degree                     312
Name: name_education_type, dtype: int64

In [424]:
# I will merge all classes containing 'SECONDARY' and 'HIGHER' 

application_cat.loc[(application_cat['name_education_type'] == 'Lower secondary') | (application_cat['name_education_type'] == 'Secondary / secondary special'), 'name_education_type'] = 'Secondary education'
application_cat.loc[(application_cat['name_education_type'] == 'Academic degree'), 'name_education_type'] = 'Higher education'
application_cat['name_education_type'].value_counts()

Secondary education    305872
Higher education       117834
Incomplete higher       14851
Name: name_education_type, dtype: int64

In [425]:
# checking frequency of classes in FAMILY STATUS
application_cat['name_family_status'].value_counts()

Married                 299828
Single / not married     55271
Civil marriage           36532
Separated                27251
Widow                    19675
Name: name_family_status, dtype: int64

In [426]:
# I will group to 2 classes 'Married' and 'Not Married'
application_cat.loc[(application_cat['name_family_status'] == 'Civil marriage'), 'name_family_status'] = 'Married'
application_cat.loc[(application_cat['name_family_status'] != 'Married'), 'name_family_status'] = 'Not Married'
application_cat['name_family_status'].value_counts()

Married        336360
Not Married    102197
Name: name_family_status, dtype: int64

In [427]:
# checking frequency of classes in HOUSING TYPE
application_cat['name_housing_type'].value_counts()

House / apartment      393831
With parents            19077
Municipal apartment     14214
Rented apartment         5974
Office apartment         3922
Co-op apartment          1539
Name: name_housing_type, dtype: int64

In [435]:
application_cat.loc[(application_cat['name_housing_type'] == 'Municipal apartment') | (application_cat['name_housing_type'] == 'Office apartment') |
                    (application_cat['name_housing_type'] == 'Office apartment') | 
                    (application_cat['name_housing_type'] == 'Co-op apartment'), 'name_housing_type'] = 'Rented apartment'

application_cat['name_housing_type'].value_counts()

House / apartment    393831
Rented apartment      25649
With parents          19077
Name: name_housing_type, dtype: int64

In [428]:
# checking frequency of classes in OCCUPATION
application_cat['occupation_type'].value_counts()

Others                   134203
Laborers                  78240
Core staff                43007
Sales staff               41098
Managers                  35487
Drivers                   26090
High skill tech staff     17289
Accountants               15985
Medicine staff            13520
Cooking staff              8076
Security staff             7993
Cleaning staff             5845
Private service staff      3456
Low-skill Laborers         2140
Secretaries                2044
Waiters/barmen staff       1665
Realty agents              1041
HR staff                    774
IT staff                    604
Name: occupation_type, dtype: int64

In [445]:
# Grouping into 3 classes: Others, Laborers, Core Staff


application_cat.loc[(application_cat['occupation_type'] == 'Drivers') | (application_cat['occupation_type'] == 'Cooking staff') | 
                    (application_cat['occupation_type'] == 'Security staff') | (application_cat['occupation_type'] == 'Cleaning staff') |
                    (application_cat['occupation_type'] == 'Private service staff') | (application_cat['occupation_type'] == 'Low-skill Laborers') |
                    (application_cat['occupation_type'] == 'Waiters/barmen staff'), 'occupation_type'] = 'Laborers'

application_cat.loc[(application_cat['occupation_type'] == 'IT staff') | (application_cat['occupation_type'] == 'Sales staff') |
                    (application_cat['occupation_type'] == 'HR staff') | (application_cat['occupation_type'] == 'Secretaries') |
                    (application_cat['occupation_type'] == 'Realty agents') | (application_cat['occupation_type'] == 'Accountants'), 'occupation_type'] = 'Laborers'

application_cat.loc[(application_cat['occupation_type'] == 'Core staff') | (application_cat['occupation_type'] == 'Managers') |
                    (application_cat['occupation_type'] == 'High skill tech staff') | 
                    (application_cat['occupation_type'] == 'Medicine staff'), 'occupation_type'] = 'Core staff'

application_cat['occupation_type'].value_counts()

/Users/steib/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/steib/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Laborers      195051
Others        134203
Core staff    109303
Name: occupation_type, dtype: int64

#### Two-sample T-Test
I want to check if my classes in Occupation makes sense: Therefore i will test if there is a difference between the incomes in the groups. 

**Assumption: if the classification is correct, than this should be reflected in the income.** I want to see, if Core Staff has higher pay than the rest. 

TWO SAMPLE RIGHT TAIL T-TEST:

-  H0: There is no difference between the income mean 
-  H1: The income mean of the Core Staff is higher
-  lambda: 0.05






In [460]:
# CHECK IF VARIANCES ARE EQUAL. IF LARGE TO SMALL RATIO IS < 4:1

# EQUALE VARIANCE:

income_core = application_records.loc[(application_cat['occupation_type'] == 'Core staff') | (application_cat['occupation_type'] == 'Managers') |
                    (application_cat['occupation_type'] == 'High skill tech staff') | 
                    (application_cat['occupation_type'] == 'Medicine staff'), 'amt_income_total']

income_others = application_records.loc[(application_cat['occupation_type'] == 'Others') | (application_cat['occupation_type'] == 'Drivers') | (application_cat['occupation_type'] == 'Cooking staff') | 
                    (application_cat['occupation_type'] == 'Security staff') | (application_cat['occupation_type'] == 'Cleaning staff') |
                    (application_cat['occupation_type'] == 'Private service staff') | (application_cat['occupation_type'] == 'Low-skill Laborers') |
                    (application_cat['occupation_type'] == 'Waiters/barmen staff') | (application_cat['occupation_type'] == 'IT staff') | (application_cat['occupation_type'] == 'Sales staff') |
                    (application_cat['occupation_type'] == 'HR staff') | (application_cat['occupation_type'] == 'Secretaries') |
                    (application_cat['occupation_type'] == 'Realty agents') | (application_cat['occupation_type'] == 'Accountants'), 'amt_income_total']

print(np.var(income_others), np.var(income_core))
print('The Ratio of both sample variances is {:.2f} which is smaller than 4:1'.format(np.var(income_core)/np.var(income_others)))
print('We can assume that the variances are the same')


# CALCULATING THE MEANS:

bar_core = round(np.mean(income_core))
bar_others = round(np.mean(income_others))

print()
print('The income mean of Core Staff {} is bigger than the income mean of the others group {}'.format(bar_core, bar_others))


# PERFORMING TWO SAMPLE T-TEST:

from scipy.stats import ttest_ind

ttest_ind(a=income_core, b=income_others, equal_var=True)

print()
print('The p-value is very small: The income seems to be signficantly different from each other')


8173956754.211781 20955220287.206646
The Ratio of both sample variances is 2.56 which is smaller than 4:1
We can assume that the variances are the same

The income mean of Core Staff 218462 is bigger than the income mean of the others group 170870

The p-value is very small: The income seems to be signficantly different from each other


#### Encoding Categorical Features

In [463]:
# Binary Features

application_cat['flag_own_car'] = application_cat['flag_own_car'].replace({'Y':1, 'N':0})

application_cat.loc[(application_cat['flag_own_realty'] ==]

/var/folders/dj/n16zrnc50j79vq_pt7btwyg40000gn/T/ipykernel_15583/2498995728.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  application_cat['flag_own_car'] = application_cat['flag_own_car'].replace({'Y':1, 'N':0})


In [462]:
application_cat

,code_gender,flag_own_car,flag_own_realty,name_income_type,name_education_type,name_family_status,name_housing_type,occupation_type
0,M,Y,Y,Working,Higher education,Married,Rented apartment,Others
1,M,Y,Y,Working,Higher education,Married,Rented apartment,Others
2,M,Y,Y,Working,Secondary education,Married,House / apartment,Laborers
3,F,N,Y,Commercial associate,Secondary education,Not Married,House / apartment,Laborers
4,F,N,Y,Commercial associate,Secondary education,Not Married,House / apartment,Laborers
...,...,...,...,...,...,...,...,...
438552,M,N,Y,Others,Secondary education,Not Married,House / apartment,Others
438553,F,N,N,Working,Secondary education,Not Married,House / apartment,Laborers
438554,F,N,N,Commercial associate,Higher education,Not Married,With parents,Laborers
438555,F,N,Y,Others,Secondary education,Married,House / apartment,Others


### Numerical Values: 

In [ ]:
application_num = application_records.select_dtypes(np.number)
application_num.head()

# NEXT STEPS: 

1) encode categorical values
2) clean numeric data

3) Check Mulitcoliniearity and clean out with VIF - Variance Inflation Factor